In [16]:
import cv2
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

import tensorflow.compat.v1 as tf

In [17]:
def LeNet(input_shape = (64, 64, 1), classes = 6):
    
    X_input = Input(input_shape)
    
    X = Conv2D(6, (3,3), strides = (1,1), name = 'conv1', kernel_initializer = glorot_uniform())(X_input)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = AveragePooling2D((3, 3))(X)
    
    X = Conv2D(16, (3,3), strides = (1,1), name = 'conv2', kernel_initializer = glorot_uniform())(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv2')(X)
    X = Activation('relu')(X)
    X = AveragePooling2D((3, 3))(X)
    
    X = Flatten()(X)
    X = Dense(120, activation='relu', name='fc2', kernel_initializer = glorot_uniform())(X)
    X = Dense(84, activation='relu', name='fc3', kernel_initializer = glorot_uniform())(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform())(X)
    
    model = Model(inputs = X_input, outputs = X, name='LeNet')

    return model

In [18]:
LeNet = LeNet(input_shape = (280, 320, 1), classes = 6)

In [19]:
LeNet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

X_train = X_train_orig/255.
X_test = X_test_orig/255.

Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

X_train1 = np.zeros((X_train.shape[0], 280, 320))
X_test1 = np.ones((X_test.shape[0], 280, 320))
rgb_weights = [0.2989, 0.5870, 0.1140]

X_train = np.dot(X_train[...,:3], rgb_weights)
for i in range(X_train.shape[0]):
    X_train1[i] = cv2.resize(X_train[i], (320,280))
X_test = np.dot(X_test[...,:3], rgb_weights)
for i in range(X_test.shape[0]):
    X_test1[i] = cv2.resize(X_test[i], (320,280))

#plt.imshow(X_train1[0], cmap='gray')
#plt.show()
#plt.imshow(X_test1[0], cmap='gray')
#plt.show()


print ("number of training examples = " + str(X_train1.shape[0]))
print ("number of test examples = " + str(X_test1.shape[0]))
print ("X_train shape: " + str(X_train1.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test1.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 280, 320)
Y_train shape: (1080, 6)
X_test shape: (120, 280, 320)
Y_test shape: (120, 6)


In [21]:
LeNet.fit(X_train1, Y_train, epochs = 25, batch_size = 32)

Epoch 1/25
34/34 [==============================] - 22s 629ms/step - loss: 2.5392 - accuracy: 0.3758
Epoch 2/25
34/34 [==============================] - 21s 608ms/step - loss: 0.4993 - accuracy: 0.8144
Epoch 3/25
34/34 [==============================] - 22s 643ms/step - loss: 0.1782 - accuracy: 0.9613
Epoch 4/25
34/34 [==============================] - 22s 650ms/step - loss: 0.1058 - accuracy: 0.9862
Epoch 5/25
34/34 [==============================] - 22s 641ms/step - loss: 0.0822 - accuracy: 0.9759
Epoch 6/25
34/34 [==============================] - 22s 636ms/step - loss: 0.0469 - accuracy: 0.9966
Epoch 7/25
34/34 [==============================] - 22s 662ms/step - loss: 0.0204 - accuracy: 1.0000
Epoch 8/25
34/34 [==============================] - 22s 646ms/step - loss: 0.0094 - accuracy: 1.0000
Epoch 9/25
34/34 [==============================] - 22s 640ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 10/25
34/34 [==============================] - 21s 631ms/step - loss: 0.0057 - accura

In [22]:
preds = LeNet.evaluate(X_test1, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 1s 122ms/step - loss: 0.2970 - accuracy: 0.9167
Loss = 0.29699742794036865
Test Accuracy = 0.9166666865348816


In [23]:
LeNet.save("LeNet_model")

INFO:tensorflow:Assets written to: LeNet_model/assets
